In [1]:
import duckdb as ddb

In [2]:
ddb.sql("INSTALL httpfs; LOAD httpfs")

In [3]:
con = ddb.connect("../air_quality.db")

In [4]:
con.execute("CREATE schema IF NOT EXISTS raw")

In [5]:
con.sql("""
    SET s3_access_key_id = '';
    SET s3_secret_access_key = '';
    SET s3_region = '';
""")

In [6]:
con.execute("""
    CREATE TABLE IF NOT EXISTS raw.air_quality_data (
        location_id BIGINT,
        sensors_id BIGINT,
        "location" VARCHAR,
        "datetime" TIMESTAMP,
        lat DOUBLE,
        lon DOUBLE,
        "parameter" VARCHAR,
        units VARCHAR,
        "value" DOUBLE,
        "month" VARCHAR,
        "year" BIGINT,
        ingestion_datetime TIMESTAMP
    );
    
""")

In [7]:
con.execute("""
INSERT INTO raw.air_quality_data
SELECT 
    location_id, 
    sensors_id, 
    "location", 
    "datetime", 
    lat, 
    lon, 
    "parameter", 
    units, 
    "value",
    "month", 
    "year",
    current_timestamp AS ingestion_datetime
FROM read_csv('s3://openaq-data-archive/records/csv.gz/locationid=3331918/year=2025/month=06/*.csv.gz');
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [8]:
con.close()